# Iris 분류

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
# data, target, target_names, feature_names, DESCR
data = iris.data
label = iris.target
columns = iris.feature_names
df = pd.DataFrame(data, columns=columns)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = \
    train_test_split(data, label, test_size=0.2, 
                     shuffle=True, stratify=label, random_state=2019)

### Logistic Regression

- 모델 객체 생성

In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(verbose=1)

- 모델 학습

In [6]:
lr.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(verbose=1)

- 학습한 결과를 저장

In [7]:
import joblib
joblib.dump(lr, 'model/iris_lr.pkl')

['model/iris_lr.pkl']

### Support Vector Machine

In [8]:
from sklearn.svm import SVC
svc = SVC()

In [9]:
svc.fit(x_train, y_train)

SVC()

In [10]:
joblib.dump(svc, 'model/iris_svm.pkl')

['model/iris_svm.pkl']

### Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [12]:
dtc.fit(x_train, y_train)

DecisionTreeClassifier()

In [13]:
joblib.dump(dtc, 'model/iris_dt.pkl')

['model/iris_dt.pkl']

### 저장된 모델로 정확도 산출

In [14]:
model_lr = joblib.load('model/iris_lr.pkl')
model_svm = joblib.load('model/iris_svm.pkl')
model_dt = joblib.load('model/iris_dt.pkl')

In [15]:
y_pred_lr = model_lr.predict(x_test)
y_pred_svm = model_svm.predict(x_test)
y_pred_dt = model_dt.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.9666666666666667 0.9666666666666667 0.9666666666666667


### 한개의 테스트 값 예측

In [17]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [18]:
test_data = np.array([6.7, 3.0, 5.2, 2.3]).reshape(1, 4)

In [19]:
index_lr = model_lr.predict(test_data)[0]
index_svm = model_svm.predict(test_data)[0]
index_dt = model_dt.predict(test_data)[0]

In [20]:
sp_names = ['Setosa', 'Versicolor', 'Virginica']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

Virginica Virginica Virginica


## Deep Learning Model

In [21]:
# 딥러닝 모델 설정
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(12, input_shape=(4,), activation='relu'),
    Dense(8, input_shape=(4,), activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                60        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [22]:
# 모델 컴파일 
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="model/iris_deep.hdf5", 
                               monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [24]:
# One hot encoding
Y_encoded = keras.utils.to_categorical(y_train)
Y_encoded[0], Y_encoded[50], Y_encoded[100]

(array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [25]:
# 모델 학습
model.fit(x_train, Y_encoded, epochs=100, batch_size=30, verbose=2,
          validation_split=0.2, callbacks=[checkpointer])

Train on 96 samples, validate on 24 samples
Epoch 1/100
 - 0s - loss: 1.1269 - accuracy: 0.3125 - val_loss: 0.9628 - val_accuracy: 0.4167

Epoch 00001: val_loss improved from inf to 0.96283, saving model to model/iris_deep.hdf5
Epoch 2/100
 - 0s - loss: 1.0713 - accuracy: 0.3125 - val_loss: 0.9357 - val_accuracy: 0.5833

Epoch 00002: val_loss improved from 0.96283 to 0.93565, saving model to model/iris_deep.hdf5
Epoch 3/100
 - 0s - loss: 1.0303 - accuracy: 0.5417 - val_loss: 0.9161 - val_accuracy: 0.7083

Epoch 00003: val_loss improved from 0.93565 to 0.91614, saving model to model/iris_deep.hdf5
Epoch 4/100
 - 0s - loss: 0.9990 - accuracy: 0.6458 - val_loss: 0.9046 - val_accuracy: 0.7083

Epoch 00004: val_loss improved from 0.91614 to 0.90463, saving model to model/iris_deep.hdf5
Epoch 5/100
 - 0s - loss: 0.9780 - accuracy: 0.6562 - val_loss: 0.8919 - val_accuracy: 0.7083

Epoch 00005: val_loss improved from 0.90463 to 0.89186, saving model to model/iris_deep.hdf5
Epoch 6/100
 - 0s - 


Epoch 00044: val_loss improved from 0.47371 to 0.47059, saving model to model/iris_deep.hdf5
Epoch 45/100
 - 0s - loss: 0.4942 - accuracy: 0.6562 - val_loss: 0.4670 - val_accuracy: 0.7083

Epoch 00045: val_loss improved from 0.47059 to 0.46702, saving model to model/iris_deep.hdf5
Epoch 46/100
 - 0s - loss: 0.4908 - accuracy: 0.6562 - val_loss: 0.4638 - val_accuracy: 0.7083

Epoch 00046: val_loss improved from 0.46702 to 0.46376, saving model to model/iris_deep.hdf5
Epoch 47/100
 - 0s - loss: 0.4875 - accuracy: 0.6562 - val_loss: 0.4608 - val_accuracy: 0.7083

Epoch 00047: val_loss improved from 0.46376 to 0.46076, saving model to model/iris_deep.hdf5
Epoch 48/100
 - 0s - loss: 0.4837 - accuracy: 0.6562 - val_loss: 0.4604 - val_accuracy: 0.7083

Epoch 00048: val_loss improved from 0.46076 to 0.46038, saving model to model/iris_deep.hdf5
Epoch 49/100
 - 0s - loss: 0.4804 - accuracy: 0.6562 - val_loss: 0.4594 - val_accuracy: 0.7083

Epoch 00049: val_loss improved from 0.46038 to 0.45943

Epoch 89/100
 - 0s - loss: 0.3149 - accuracy: 0.9688 - val_loss: 0.2928 - val_accuracy: 1.0000

Epoch 00089: val_loss improved from 0.29734 to 0.29277, saving model to model/iris_deep.hdf5
Epoch 90/100
 - 0s - loss: 0.3115 - accuracy: 0.9688 - val_loss: 0.2924 - val_accuracy: 1.0000

Epoch 00090: val_loss improved from 0.29277 to 0.29243, saving model to model/iris_deep.hdf5
Epoch 91/100
 - 0s - loss: 0.3093 - accuracy: 0.9688 - val_loss: 0.2891 - val_accuracy: 1.0000

Epoch 00091: val_loss improved from 0.29243 to 0.28907, saving model to model/iris_deep.hdf5
Epoch 92/100
 - 0s - loss: 0.3052 - accuracy: 0.9688 - val_loss: 0.2812 - val_accuracy: 1.0000

Epoch 00092: val_loss improved from 0.28907 to 0.28124, saving model to model/iris_deep.hdf5
Epoch 93/100
 - 0s - loss: 0.3040 - accuracy: 0.9583 - val_loss: 0.2737 - val_accuracy: 1.0000

Epoch 00093: val_loss improved from 0.28124 to 0.27373, saving model to model/iris_deep.hdf5
Epoch 94/100
 - 0s - loss: 0.3017 - accuracy: 0.9583 - 

In [26]:
from keras.models import load_model
model_deep = load_model("model/iris_deep.hdf5")

In [27]:
model.evaluate(x_test, keras.utils.to_categorical(y_test))[1]

30/30 [==============================] - 0s 33us/step


1.0

In [28]:
model_deep.predict_classes(test_data)[0]

2

In [29]:
model_deep.predict(test_data)

array([[0.00281011, 0.2834836 , 0.71370625]], dtype=float32)

In [30]:
np.argmax(model_deep.predict(test_data)[0])

2